In [1]:
import pandas as pd

In [5]:
# get cohort patients
cohort_id = pd.read_csv('/Volumes/D/F2023/MIM/mimic_processed/cohort_id.csv', index_col=[0])

In [8]:
cohort_patients = pd.read_csv('/Volumes/D/F2023/mimic-iv-2.2/hosp/patients.csv')[pd.read_csv('/Volumes/D/F2023/mimic-iv-2.2/hosp/patients.csv').subject_id.isin(cohort_id.subject_id)]

In [9]:
cohort_patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
33,10000980,F,73,2186,2008 - 2010,2193-08-26
65,10002013,F,53,2156,2008 - 2010,NaN
103,10002930,F,48,2193,2011 - 2013,2201-12-24
116,10003400,F,72,2134,2011 - 2013,2137-09-02
135,10004401,M,82,2141,2008 - 2010,2144-06-18
...,...,...,...,...,...,...
299631,19997293,M,76,2123,2011 - 2013,2124-02-20
299659,19998330,F,71,2177,2011 - 2013,2178-12-08
299669,19998591,F,52,2184,2008 - 2010,2185-12-05
299677,19998878,M,56,2132,2008 - 2010,2133-01-03


In [12]:
cohort_admission = pd.read_csv('/Volumes/D/F2023/mimic-iv-2.2/hosp/admissions.csv')[pd.read_csv('/Volumes/D/F2023/mimic-iv-2.2/hosp/admissions.csv').subject_id.isin(cohort_id.subject_id)]

In [14]:
cohort_admission['admittime'] = pd.to_datetime(cohort_admission['admittime'])
cohort_admission['dischtime'] = pd.to_datetime(cohort_admission['dischtime'])

In [16]:
def length_of_stay(x):
    difference = x['dischtime'] - x['admittime']
    by_hours = difference.total_seconds() / 3600
    formatted = "{:.2f}".format(by_hours)
    return formatted

In [25]:
cohort_admission['length_of_stay'] = (cohort_admission.apply(lambda x: length_of_stay(x), axis=1)).astype(float)

In [85]:
cohort_adm_stay_aggr = cohort_admission.groupby('subject_id').agg(
    num_of_visits = pd.NamedAgg(column='hadm_id', aggfunc='count'),
    total_length_of_stay=pd.NamedAgg(column='length_of_stay', aggfunc='sum')
).reset_index(drop=False)

In [41]:
cohort_earliest_adm = cohort_admission.groupby('subject_id').agg(
    earliest = pd.NamedAgg(column='admittime', aggfunc='min')
).reset_index(drop=False)
cohort_earliest_adm

,subject_id,earliest
0,10000980,2188-01-03 17:41:00
1,10002013,2156-06-28 15:53:00
2,10002930,2193-08-05 06:18:00
3,10003400,2134-06-06 02:25:00
4,10004401,2141-06-06 19:20:00
...,...,...
11587,19997293,2123-10-12 13:46:00
11588,19998330,2177-07-25 04:34:00
11589,19998591,2185-03-10 14:19:00
11590,19998878,2132-03-31 06:39:00


In [98]:
def earliest_info(dataframe):
    output = []

    # gather unique ID
    pid_list = dataframe.subject_id.unique()

    for pid in pid_list:
        df = dataframe[dataframe.subject_id == pid]
        sorted = df.sort_values('admittime')
        output.append(sorted.head(1)[['subject_id', 'admittime', 'dischtime','deathtime','admission_type', 'admission_location','discharge_location', 'insurance','race']])
    return output

def last_info(dataframe):
    output = []

    # gather unique ID
    pid_list = dataframe.subject_id.unique()

    for pid in pid_list:
        df = dataframe[dataframe.subject_id == pid]
        sorted = df.sort_values('admittime')
        output.append(sorted.tail(1)[['subject_id','admittime', 'dischtime','deathtime','admission_type', 'admission_location','discharge_location', 'insurance','race']])
    return output


In [99]:
earliest_list = earliest_info(dataframe=cohort_admission)
last_list = last_info(dataframe=cohort_admission)

In [101]:
pd.concat(earliest_list, axis=0)

,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,race
33,10000980,2188-01-03 17:41:00,2188-01-05 17:30:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,BLACK/AFRICAN AMERICAN
97,10002013,2156-06-28 15:53:00,2156-07-02 15:39:00,NaN,EW EMER.,PACU,HOME,Medicaid,WHITE
152,10002930,2193-08-05 06:18:00,2193-08-05 11:44:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicare,BLACK/AFRICAN AMERICAN
196,10003400,2134-06-06 02:25:00,2134-06-07 15:05:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,BLACK/AFRICAN AMERICAN
228,10004401,2141-06-06 19:20:00,2141-06-09 12:34:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicare,WHITE
...,...,...,...,...,...,...,...,...,...
431101,19997293,2123-10-12 13:46:00,2123-10-30 15:13:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,WHITE
431147,19998330,2177-07-25 04:34:00,2177-07-26 14:07:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,BLACK/AFRICAN AMERICAN
431167,19998591,2185-03-10 14:19:00,2185-03-11 14:10:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,WHITE
431181,19998878,2132-03-31 06:39:00,2132-04-02 16:15:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicare,WHITE


In [77]:
merged1 = cohort_earliest_adm.merge(pd.concat(earliest_list, axis=0), on='subject_id', how='left')

In [103]:
merged11 = merged1.merge(pd.concat(last_list, axis=0).drop('race', axis=1), on='subject_id', how='left')
merged11

,subject_id,earliest,deathtime_x,admission_type_x,admission_location_x,discharge_location_x,insurance_x,race,admittime,dischtime,deathtime_y,admission_type_y,admission_location_y,discharge_location_y,insurance_y
0,10000980,2188-01-03 17:41:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,BLACK/AFRICAN AMERICAN,2193-08-15 01:01:00,2193-08-17 15:07:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other
1,10002013,2156-06-28 15:53:00,NaN,EW EMER.,PACU,HOME,Medicaid,WHITE,2167-07-05 06:10:00,2167-07-05 11:44:00,NaN,EU OBSERVATION,PHYSICIAN REFERRAL,NaN,Other
2,10002930,2193-08-05 06:18:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Medicare,BLACK/AFRICAN AMERICAN,2201-03-23 19:15:00,2201-03-26 14:24:00,NaN,EU OBSERVATION,PHYSICIAN REFERRAL,NaN,Medicare
3,10003400,2134-06-06 02:25:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,BLACK/AFRICAN AMERICAN,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare
4,10004401,2141-06-06 19:20:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicare,WHITE,2144-06-05 19:45:00,2144-06-18 21:30:00,2144-06-18 21:30:00,EW EMER.,EMERGENCY ROOM,DIED,Medicare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11587,19997293,2123-10-12 13:46:00,NaN,URGENT,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,WHITE,2123-12-29 01:47:00,2124-01-10 16:30:00,NaN,URGENT,TRANSFER FROM HOSPITAL,CHRONIC/LONG TERM ACUTE CARE,Medicare
11588,19998330,2177-07-25 04:34:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,BLACK/AFRICAN AMERICAN,2178-11-27 21:51:00,2178-12-01 17:10:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Other
11589,19998591,2185-03-10 14:19:00,NaN,EU OBSERVATION,EMERGENCY ROOM,NaN,Other,WHITE,2185-07-03 20:20:00,2185-08-03 14:42:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Medicare
11590,19998878,2132-03-31 06:39:00,NaN,EW EMER.,EMERGENCY ROOM,HOME,Medicare,WHITE,2132-12-20 06:00:00,2132-12-26 13:45:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare


In [104]:
merged2 = merged11.merge(cohort_adm_stay_aggr, on='subject_id',how='left')

In [105]:
merged3 = cohort_patients.merge(merged2, on='subject_id', how='left')

In [108]:
merged3[['deathtime_x','deathtime_y']]

,deathtime_x,deathtime_y
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,2137-09-02 17:05:00
4,NaN,2144-06-18 21:30:00
...,...,...
11587,NaN,NaN
11588,NaN,NaN
11589,NaN,NaN
11590,NaN,NaN


In [111]:

merged3.columns = ['subject_id', 'gender', 'anchor_age', 'anchor_year',
                    'anchor_year_group', 'dod', 'earliest', 'deathtime_x',
                    'admission_type_x', 'admission_location_x', 'discharge_location_x',
                    'insurance_x', 'race', 'admittime_y', 'dischtime_y', 'deathtime_y','admission_type_y', 'admission_location_y', 'discharge_location_y','insurance_y', 'num_of_visits', 'total_length_of_stay']


In [112]:
merged3

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,earliest,deathtime_x,admission_type_x,admission_location_x,...,race,admittime_y,dischtime_y,deathtime_y,admission_type_y,admission_location_y,discharge_location_y,insurance_y,num_of_visits,total_length_of_stay
0,10000980,F,73,2186,2008 - 2010,2193-08-26,2188-01-03 17:41:00,NaN,EW EMER.,EMERGENCY ROOM,...,BLACK/AFRICAN AMERICAN,2193-08-15 01:01:00,2193-08-17 15:07:00,NaN,OBSERVATION ADMIT,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,7,578.24
1,10002013,F,53,2156,2008 - 2010,NaN,2156-06-28 15:53:00,NaN,EW EMER.,PACU,...,WHITE,2167-07-05 06:10:00,2167-07-05 11:44:00,NaN,EU OBSERVATION,PHYSICIAN REFERRAL,NaN,Other,12,943.30
2,10002930,F,48,2193,2011 - 2013,2201-12-24,2193-08-05 06:18:00,NaN,EU OBSERVATION,EMERGENCY ROOM,...,BLACK/AFRICAN AMERICAN,2201-03-23 19:15:00,2201-03-26 14:24:00,NaN,EU OBSERVATION,PHYSICIAN REFERRAL,NaN,Medicare,12,958.43
3,10003400,F,72,2134,2011 - 2013,2137-09-02,2134-06-06 02:25:00,NaN,EW EMER.,EMERGENCY ROOM,...,BLACK/AFRICAN AMERICAN,2137-08-04 00:07:00,2137-09-02 17:05:00,2137-09-02 17:05:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare,7,1971.83
4,10004401,M,82,2141,2008 - 2010,2144-06-18,2141-06-06 19:20:00,NaN,EW EMER.,EMERGENCY ROOM,...,WHITE,2144-06-05 19:45:00,2144-06-18 21:30:00,2144-06-18 21:30:00,EW EMER.,EMERGENCY ROOM,DIED,Medicare,12,1626.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11587,19997293,M,76,2123,2011 - 2013,2124-02-20,2123-10-12 13:46:00,NaN,URGENT,TRANSFER FROM HOSPITAL,...,WHITE,2123-12-29 01:47:00,2124-01-10 16:30:00,NaN,URGENT,TRANSFER FROM HOSPITAL,CHRONIC/LONG TERM ACUTE CARE,Medicare,4,1053.64
11588,19998330,F,71,2177,2011 - 2013,2178-12-08,2177-07-25 04:34:00,NaN,EW EMER.,EMERGENCY ROOM,...,BLACK/AFRICAN AMERICAN,2178-11-27 21:51:00,2178-12-01 17:10:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Other,6,589.91
11589,19998591,F,52,2184,2008 - 2010,2185-12-05,2185-03-10 14:19:00,NaN,EU OBSERVATION,EMERGENCY ROOM,...,WHITE,2185-07-03 20:20:00,2185-08-03 14:42:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,2,762.22
11590,19998878,M,56,2132,2008 - 2010,2133-01-03,2132-03-31 06:39:00,NaN,EW EMER.,EMERGENCY ROOM,...,WHITE,2132-12-20 06:00:00,2132-12-26 13:45:00,NaN,EW EMER.,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,6,521.47


In [59]:
cohort_race_ml = cohort_admission[['subject_id', 'race', 'length_of_stay']].pivot_table(index='subject_id', columns='race', aggfunc='count', fill_value=0).reset_index(drop=False)

In [58]:
cohort_admission.race.value_counts()

race
WHITE                                        39291
BLACK/AFRICAN AMERICAN                       13530
OTHER                                         1691
HISPANIC/LATINO - PUERTO RICAN                1593
HISPANIC OR LATINO                            1593
WHITE - RUSSIAN                               1117
UNKNOWN                                       1037
WHITE - OTHER EUROPEAN                         747
ASIAN - CHINESE                                640
HISPANIC/LATINO - DOMINICAN                    625
BLACK/CAPE VERDEAN                             600
BLACK/CARIBBEAN ISLAND                         597
ASIAN                                          534
PORTUGUESE                                     425
UNABLE TO OBTAIN                               320
BLACK/AFRICAN                                  317
ASIAN - SOUTH EAST ASIAN                       312
HISPANIC/LATINO - GUATEMALAN                   207
PATIENT DECLINED TO ANSWER                     191
ASIAN - ASIAN INDIAN      

In [76]:
cohort_admission

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,length_of_stay
28,10000980,20897796,2193-08-15 01:01:00,2193-08-17 15:07:00,NaN,OBSERVATION ADMIT,P77SO2,WALK-IN/SELF REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2193-08-14 21:25:00,2193-08-15 02:22:00,0,62.10
29,10000980,24947999,2190-11-06 20:57:00,2190-11-08 15:58:00,NaN,EW EMER.,P434W4,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2190-11-06 15:30:00,2190-11-06 23:16:00,0,43.02
30,10000980,25242409,2191-04-03 18:48:00,2191-04-11 16:21:00,NaN,EW EMER.,P33K8A,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2191-04-03 12:36:00,2191-04-03 20:29:00,0,189.55
31,10000980,25911675,2191-05-23 15:33:00,2191-05-24 17:14:00,NaN,EW EMER.,P29CGZ,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2191-05-22 16:06:00,2191-05-23 17:56:00,0,25.68
32,10000980,26913865,2189-06-27 07:38:00,2189-07-03 03:00:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,2189-06-27 06:25:00,2189-06-27 08:42:00,0,139.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431182,19998878,27930413,2132-05-23 16:10:00,2132-05-27 19:00:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicare,ENGLISH,SINGLE,WHITE,2132-05-23 14:30:00,2132-05-23 17:51:00,0,98.83
431183,19998878,29356037,2132-08-17 20:59:00,2132-08-23 14:00:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,SINGLE,WHITE,2132-08-17 14:32:00,2132-08-17 21:33:00,0,137.02
431190,19999287,20175828,2197-08-03 20:58:00,2197-08-18 15:37:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOSPICE,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2197-08-03 19:34:00,2197-08-04 00:02:00,0,354.65
431191,19999287,22997012,2197-07-26 03:29:00,2197-07-31 14:00:00,NaN,EW EMER.,P44WVR,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2197-07-26 02:51:00,2197-07-26 05:00:00,0,130.52
